In [24]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(stringr))
library(parallel)
library(fgsea)
library(data.table)
source('/home//jupyter/BRI_Figures_Final_V2/helper_function/helper_function_IHA.r')

In [25]:
meta_data=read.csv("/home/jupyter/BRI_Figures_Final_V2/Figure5/01_Frequency_Comparison/selected_samples_with_acutal_flu_year.csv")

In [26]:
file_list<-paste0("/home/jupyter/BRI_Figures_Final_V2/Dataset/scRNA/BRI/Average_LogNormalized_Expression/Average_LogNormalized_Expression_of_Celltypes_by_Sample_AIFI_L3/",meta_data$pbmc_sample_id,".csv")

df_list<-read_pseudobulk_expression(file_list)

[1] "Total reading time: 17.898 seconds"
[1] "The length of the list matches the length of the input path."


In [27]:
genes_of_interest <- c( "CD19", "TBX21", "ZEB2", "MS4A1", "ITGAX", "XBP1", "DAPP1", "FCRL5", "BATF","IGHM", "IGHD", "IGHG1", "IGHG2", "IGHG3", "IGHG4")



In [28]:
length(genes_of_interest)

[1] 15

In [29]:
  
celltype_list <- lapply(df_list, function(df) { df <- df[genes_of_interest, grep('Core memory B cell', names(df), fixed = TRUE),drop=FALSE] })


In [30]:
exp_matrix<-do.call(cbind,celltype_list)

colnames(exp_matrix)<-sub(":.*", "", colnames(exp_matrix))
rownames(meta_data)<-meta_data$pbmc_sample_id

In [31]:
exp_matrix<-exp_matrix[meta_data$pbmc_sample_id]

In [32]:
rownames(exp_matrix)<-paste0("Gene_",rownames(exp_matrix))

In [33]:
df<-cbind(meta_data,t(exp_matrix))

In [34]:

averaged_df <- df %>%
  group_by(Flu_Year, Flu_Day, cohort.cohortGuid) %>%
  summarise(across(starts_with("Gene"), mean))

`summarise()` has grouped output by 'Flu_Year', 'Flu_Day'. You can override
using the `.groups` argument.


In [35]:
averaged_df<-averaged_df %>% arrange(cohort.cohortGuid,Flu_Year,Flu_Day)

In [36]:
write.csv(averaged_df,"selected_gene_CoreB.csv")
